# Time Series Deep Learning Workshop − Exercise

ODSC East − April 15, 2020

Mark Steadman, Viktor Kovryzhkin

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt

### Helper functions 
Functions for creating sliding windows, splitting data into train/test partitions, metrics, etc.

In [ ]:
def sliding_window(arr, window_size):
    """ Create sliding windows on array 
    
    Parameters
    ----------
    arr: np.ndarray
        array to reshape
    window_size: int
        sliding window size
    Returns
    -------
    new_arr: np.ndarray
        2D matrix of shape (arr.shape[0] - window_size + 1, window_size)
    """
    (stride,) = arr.strides
    arr = np.lib.index_tricks.as_strided(
        arr,
        (arr.shape[0] - window_size + 1, window_size),
        strides=[stride, stride],
        writeable=False,
    )
    return arr


def prepare_data(y, fdw, fw, test_size=0.1, take=None):
    """ General data preparation function. 
    Takes array of targets and does following steps: 
    1) Removes nan valies 
    2) Creates sliding windows using feature derivation and forecast windows
    3) Splits data into train and test partitions
    
    Parameters
    ----------
    y: np.ndarray
        Target series 
    fdw: int 
        Feature derivation window 
    fw: int 
        Forecast window 
    test_size: float 
        Size of the validation partition
    take: int
        Number of rows to take from original array. Useful for fast experiments of dataset is large. 
        
    Returns
    -------
    x_train: np.ndarray
        training X matrix of shape (?, FDW, 1)
    y_train: np.ndarray
        training y matrix of shape (?, FW)
    x_test: np.ndarray
        testing X matrix of shape (?, FDW, 1)
    y_test: np.ndarray
        testing y matrix of shape (?, FW, 1)
    """
    y = y[~pd.isnull(y)]
    
    if take is not None: 
        y = y[-take:]
    
    windows = sliding_window(y, fdw + fw)
        
    num_test_batches = int(windows.shape[0] * test_size)
    train_windows = windows[:-num_test_batches]
    test_windows = windows[-num_test_batches:]

    x_train = train_windows[:, :fdw, np.newaxis]
    y_train = train_windows[:, fdw:]

    x_test = test_windows[:, :fdw, np.newaxis]
    y_test = test_windows[:, fdw:]

    return x_train, y_train, x_test, y_test
    

def simple_baseline(x, y):
    """ Simple naive baseline predictions. 
    For each row in y predicts latest known value. 
    """
    latest = x[:, -1, :]
    y_pred = np.tile(latest, (1, y.shape[1]))
    return y_pred


def seasonal_baseline(x, y, period): 
    """ Seasonal naive baseline predictions. 
    """
    indexes = np.arange(y.shape[1]) - period
    return np.squeeze(x_test[:, indexes])


def mase(y_true, y_pred, y_naive_pred):
    baseline_mae = mean_absolute_error(y_true, y_naive_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mae / baseline_mae


def rmse(y_true, y_pred): 
    return np.sqrt(mean_squared_error(y_true, y_pred))


def plot_predictions(x, y_true, y_pred): 
    x_indexes = np.arange(x.shape[0])
    y_indexes = np.arange(x.shape[0], x.shape[0] + y_true.shape[0])
    line1 = plt.plot(x_indexes, x)
    line2 = plt.plot(y_indexes, y_true)
    line3 = plt.plot(y_indexes, y_pred)
    
    line1[0].set_label('Historical data')
    line2[0].set_label('Ground truth')
    line3[0].set_label('Forecast')
    plt.legend()

    
def plot_loss(history): 
    loss = history['loss']
    line = plt.plot(range(len(loss)), loss)
    line[0].set_label('Training loss')
    if 'val_loss' in history: 
        val_loss = history['val_loss']
        line = plt.plot(range(len(val_loss)), val_loss)
        line[0].set_label('Validation loss')
    plt.legend()
    
    
class ScalerWrapper(object):
    """ Helped wrapper class to help scale multi dimentional matrices usings scale.  
    """
    def __init__(self, scaler):
        self.scaler = scaler 
        
    def fit(self, X): 
        self.scaler.fit(np.reshape(X, (-1, 1)))
        return self
    
    def transform(self, X):
        original_shape = X.shape
        X_scaled = self.scaler.transform(np.reshape(X, (-1, 1)))
        return np.reshape(X_scaled, original_shape)
    
    def inverse_transform(self, X):
        original_shape = X.shape
        X_unscaled = self.scaler.inverse_transform(np.reshape(X, (-1, 1)))
        return np.reshape(X_unscaled, original_shape)
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)
    

### Example dataset

NYC hourly energy consumption dataset.  
Has strong daily seasonality and weekly seasonality

In [ ]:
df = pd.read_csv('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/nyc_energy.csv')
plt.figure(figsize=(10, 5))
plt.plot(df.index[:500], df['demand'][:500])

#### Setup 

Dataset has weekly seasonality, that's why in order to capture it we need to learn from the whole week.   
Thus we will be using feature derivation window equal to 168 time steps (168 hours in a week) and forecast window of 24 hours to predict for the next day. 

In [ ]:
# size of the feature derivation window or number of time steps to look back 
FDW = 168
# size of the forecast window or number of time steps to predict into the future 
FW = 24
# size of the test set 
test_size = 0.1
# number of rows use for training and testing. Set this to None to use entire dataset (~40k rows)
nrows = 5000 

# name of the target column in csv file
target = 'demand'

In [ ]:
# split data into train and test and create sliding windows 

x_train, y_train, x_test, y_test = prepare_data(df[target].values, fdw=FDW, fw=FW, test_size=test_size, take=nrows)

Here is how one of the sliding window looks like.  

In [ ]:
plt.figure(figsize=(10, 5))
train_line = plt.plot(np.arange(FDW), x_train[0])
train_line[0].set_label('Historical data')
test_line = plt.plot(np.arange(FDW, FDW + FW), y_train[0])
test_line[0].set_label('Future data')
plt.legend()

### First, let's evaluate baselines 

Simple naive baseline outputs latest known value, i.e. last value in historical sequence.  

In [ ]:
y_simple_pred = simple_baseline(x_test, y_test)
print('Simple baseline')
print('\tRMSE {}'.format(rmse(y_test, y_simple_pred)))
print('\tMAE  {}'.format(mean_absolute_error(y_test, y_simple_pred)))

plot_predictions(x_test[0], y_test[0], y_simple_pred[0])

Dataset has daily seasonality, i.e. the pattern repeats every day - enery consumption rises during the day and decreases during the night.  
In this case baseline which outputs latest value for the same hour of previous day will be better.  
For example energy consumption at 5PM today will likely be more similar to energy consumption 5PM yesterday, rather then an hour ago. 

In [ ]:
y_24h_pred = seasonal_baseline(x_test, y_test, period=24)
print('24h seasonal baseline')
print('\tRMSE {}'.format(rmse(y_test, y_24h_pred)))
print('\tMAE  {}'.format(mean_absolute_error(y_test, y_24h_pred)))

plot_predictions(x_test[0], y_test[0], y_24h_pred[0])

Dataset also exposes weekly seasonality with some repeating pattern on weekends.  
Thus another baseline worth comparing is baseline which outputs prediction for the same hour of day last day of week.  
In another words 1PM on Saturday will likely be more similar to 1PM previos Saturday rather then 1PM Friday.

In [ ]:
y_168h_pred = seasonal_baseline(x_test, y_test, period=168)
print('168h (1week) seasonal baseline')
print('\tRMSE {}'.format(rmse(y_test, y_168h_pred)))
print('\tMAE  {}'.format(mean_absolute_error(y_test, y_168h_pred)))

plot_predictions(x_test[0], y_test[0], y_168h_pred[0])

### Neural nets 

Now it is time to train some models.  

There are two very simple LSTM based models avaliable here. 
1) SimpleLSTMEstimator - network with LSTM encoder and dense layer which outputs predictions for all time steps into the future.  
2) SimpleSeq2SeqLSTMEstimator - network which uses LSTM layers for both encoder and decoder in a way similar to sequence-to-sequence architecture. This architecture doesn't use rolling structure of the decoder prediction, just passes last encoder output into decoder at every time step.

In [ ]:
class SimpleLSTMEstimator(object): 
    def __init__(self, fdw, fw, lstm_units=None, scaler=None): 
        self.fdw = fdw
        self.fw = fw
        self.lstm_units = lstm_units
        self.scaler = scaler 

    def build_model(self): 
        inputs = tf.keras.Input(shape=[self.fdw, 1], name='inputs')
        
        rnn_cells = []
        for units in self.lstm_units:
            cell = tf.keras.layers.LSTMCell(units)
            rnn_cells.append(cell)
        outputs = tf.keras.layers.RNN(rnn_cells)(inputs)
        
        outputs = tf.keras.layers.Dense(self.fw, activation='linear')(outputs)
        
        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
    def fit(self, X, y, optimizer='adam', loss='mae', **kwargs): 
        if self.scaler: 
            X = self.scaler.fit_transform(X)
            y = self.scaler.transform(y)

        self.build_model()
        self.model.compile(optimizer=optimizer, loss=loss)
        
        
        return self.model.fit(X, y, **kwargs)
        
    def predict(self, X, **kwargs): 
        if self.scaler: 
            X = self.scaler.transform(X)

        predictions = self.model.predict(X)
        
        if self.scaler: 
            predictions = self.scaler.inverse_transform(predictions)        
        return predictions

    
class SimpleSeq2SeqLSTMEstimator(object): 
    
    def __init__(self, fdw, fw, lstm_units=None, scaler=None): 
        self.fdw = fdw
        self.fw = fw
        self.lstm_units = lstm_units
        self.scaler = scaler 
        
    def build_model(self): 
        inputs = tf.keras.Input(shape=[self.fdw, 1], name='inputs')
        
        encoder_cells = []
        for units in self.lstm_units:
            cell = tf.keras.layers.LSTMCell(units)
            encoder_cells.append(cell)
        outputs = tf.keras.layers.RNN(encoder_cells)(inputs)
        
        outputs = tf.keras.layers.RepeatVector(self.fw)(outputs)

        decoder_cells = []
        for units in self.lstm_units:
            cell = tf.keras.layers.LSTMCell(units)
            decoder_cells.append(cell)
        outputs = tf.keras.layers.RNN(decoder_cells, return_sequences=True)(outputs)
        
        outputs = tf.keras.layers.Dense(1, activation='linear')(outputs)
        
        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        
    def fit(self, X, y, optimizer='adam', loss='mae', **kwargs): 
        if self.scaler: 
            X = self.scaler.fit_transform(X)
            y = self.scaler.transform(y)
            
        self.build_model()
        self.model.compile(optimizer=optimizer, loss=loss)
        return self.model.fit(X, y, **kwargs)
        
    def predict(self, X, **kwargs): 
        if self.scaler: 
            X = self.scaler.transform(X)
            
        predictions = self.model.predict(X)
        
        if self.scaler: 
            predictions = self.scaler.inverse_transform(predictions)                
        return predictions

In [ ]:
# Training setup 
epochs = 10

batch_size = 256

validation_split = 0.1

# 1 for verbose training output (loss and val_loss for each epoch)
verbose = 0

# Number of LSTM units. Add elements to the list to create stacked LSTMs. 
lstm_units = [64] 

In [ ]:
estimator = SimpleLSTMEstimator(
    fdw=FDW,
    fw=FW,
    lstm_units=lstm_units,
    scaler=ScalerWrapper(StandardScaler()),
)
history = estimator.fit(
    x_train, y_train, 
    optimizer=tf.keras.optimizers.Adam(0.01), 
    loss='mae', 
    shuffle=True, 
    epochs=epochs, 
    batch_size=batch_size,
    verbose=verbose,
    validation_split=validation_split,
)
plot_loss(history.history)

In [ ]:
seq2seq = SimpleSeq2SeqLSTMEstimator(
    fdw=FDW,
    fw=FW,
    lstm_units=lstm_units,
    scaler=ScalerWrapper(StandardScaler()),
)
history = seq2seq.fit(
    x_train, y_train, 
    optimizer=tf.keras.optimizers.Adam(0.01), 
    loss='mae', 
    shuffle=True, 
    epochs=epochs, 
    batch_size=batch_size,
    verbose=verbose,
    validation_split=validation_split,
)
plot_loss(history.history)

In [ ]:
i = np.random.randint(x_test.shape[0])

In [ ]:
y_pred = estimator.predict(x_test)
y_pred = np.squeeze(y_pred)
print('Simple LSTM model')
print('\tRMSE        {}'.format(rmse(y_test, y_pred)))
print('\tMAE         {}'.format(mean_absolute_error(y_test, y_pred)))
print('\tMASE latest {}'.format(mase(y_test, y_pred, y_simple_pred)))
print('\tMASE 24h    {}'.format(mase(y_test, y_pred, y_24h_pred)))
print('\tMASE 168h   {}'.format(mase(y_test, y_pred, y_168h_pred)))
plot_predictions(x_test[i], y_test[i], y_pred[i])

In [ ]:
y_pred = seq2seq.predict(x_test)
y_pred = np.squeeze(y_pred)
print('Simple LSTM model')
print('\tRMSE        {}'.format(rmse(y_test, y_pred)))
print('\tMAE         {}'.format(mean_absolute_error(y_test, y_pred)))
print('\tMASE latest {}'.format(mase(y_test, y_pred, y_simple_pred)))
print('\tMASE 24h    {}'.format(mase(y_test, y_pred, y_24h_pred)))
print('\tMASE 168h   {}'.format(mase(y_test, y_pred, y_168h_pred)))
plot_predictions(x_test[i], y_test[i], y_pred[i])

## Another Dataset &minus; Stock Data

In [ ]:
df = pd.read_csv('data/stock_close_prices.csv')
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['GOOG'])

In [ ]:
FDW = 30
FW = 7
test_size = 0.2
target = 'GOOG'

In [ ]:
# split data into train and test and create sliding windows 
x_train, y_train, x_test, y_test = prepare_data(df[target].values, fdw=FDW, fw=FW, test_size=test_size)

Let's use simple latest baseline for this dataset cause it doesn't expose really strong seasonal pattern 

In [ ]:
y_simple_pred = simple_baseline(x_test, y_test)
print('Simple baseline')
print('\tRMSE {}'.format(rmse(y_test, y_simple_pred)))
print('\tMAE  {}'.format(mean_absolute_error(y_test, y_simple_pred)))

plot_predictions(x_test[0], y_test[0], y_simple_pred[0])